# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15, readonly=True)

<IB connected to 127.0.0.1:7497 clientId=15>

In [43]:
contract = Stock('INTC', 'SMART', 'USD')
ib.qualifyContracts(contract)
ib.reqMarketDataType(1)
ib.reqMktData(contract, '100,101,104,105,106,258,456', False, False)

ticker = ib.ticker(contract)

ib.sleep(1)

ticker.dict()

{'contract': Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'),
 'time': datetime.datetime(2020, 9, 22, 15, 26, 5, 970989, tzinfo=datetime.timezone.utc),
 'marketDataType': 1,
 'bid': 49.54,
 'bidSize': 23,
 'ask': 49.55,
 'askSize': 111,
 'last': 49.54,
 'lastSize': 1,
 'prevBid': nan,
 'prevBidSize': 15,
 'prevAsk': nan,
 'prevAskSize': 113,
 'prevLast': nan,
 'prevLastSize': nan,
 'volume': 76756,
 'open': 49.9,
 'high': 50.21,
 'low': 49.51,
 'close': 49.72,
 'vwap': nan,
 'low13week': nan,
 'high13week': nan,
 'low26week': nan,
 'high26week': nan,
 'low52week': nan,
 'high52week': nan,
 'bidYield': nan,
 'askYield': nan,
 'lastYield': nan,
 'markPrice': nan,
 'halted': 0.0,
 'rtHistVolatility': nan,
 'rtVolume': nan,
 'rtTradeVolume': nan,
 'avVolume': nan,
 'tradeCount': nan,
 'tradeRate': nan,
 'volumeRate': nan,
 'shortableShares': nan,
 'indexFuturePremium': nan,
 'futuresOpenInterest': nan,
 

Ticker(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), time=datetime.datetime(2020, 9, 22, 9, 8, 32, 707607, tzinfo=datetime.timezone.utc), bid=49.84, bidSize=4, ask=49.93, askSize=1, last=49.85, lastSize=1, volume=43, close=49.72, halted=0.0, ticks=[TickData(time=datetime.datetime(2020, 9, 22, 9, 8, 32, 707607, tzinfo=datetime.timezone.utc), tickType=1, price=49.84, size=4), TickData(time=datetime.datetime(2020, 9, 22, 9, 8, 32, 707607, tzinfo=datetime.timezone.utc), tickType=2, price=49.93, size=1), TickData(time=datetime.datetime(2020, 9, 22, 9, 8, 32, 707607, tzinfo=datetime.timezone.utc), tickType=4, price=49.85, size=1), TickData(time=datetime.datetime(2020, 9, 22, 9, 8, 32, 707607, tzinfo=datetime.timezone.utc), tickType=5, price=49.85, size=1), TickData(time=datetime.datetime(2020, 9, 22, 9, 8, 32, 707607, tzinfo=datetime.timezone.utc), tickType=8, price=-1.0, size=43), TickData(tim

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [7]:
ticker.avOptionVolume

nan

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [7]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,1000000,1.17385,1.17386,2000000,1.17735,1.172,1.1772
USDJPY,3000000,104.56,104.563,2100000,104.755,104.465,104.65
GBPUSD,2000000,1.28159,1.28165,2500000,1.28385,1.27125,1.2816
USDCHF,1000000,0.91624,0.91625,1000000,0.91735,0.91415,0.91455
USDCAD,4000000,1.33115,1.33121,2000000,1.3346,1.32975,1.3305
AUDUSD,1000000,0.72069,0.72071,4000000,0.7235,0.7178,0.7223


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [8]:
for contract in contracts:
    ib.cancelMktData(contract)

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [9]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2020, 9, 22, 8, 50, 53, 46078, tzinfo=datetime.timezone.utc), bid=1.17402, bidSize=1000000, ask=1.17403, askSize=6000000, prevBid=1.17399, prevBidSize=2000000, prevAsk=1.17401, prevAskSize=3000000, high=1.17735, low=1.172, close=1.1772, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2020, 9, 22, 8, 50, 53, 46078, tzinfo=datetime.timezone.utc), bidPrice=1.17402, askPrice=1.17403, bidSize=1000000, askSize=6000000, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False))])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [10]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2020, 9, 22, 8, 50, 56, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.17391, priceAsk=1.17392, sizeBid=4000000, sizeAsk=1000000)

In [10]:
ib.disconnect()